In [3]:
import tensorflow as tf
import larq as lq
import numpy as np
import pandas as pd
import seaborn as sns
from binaryflow import quantizers
from binaryflow.layers import ABCNet,XnorNet,BinaryNet
from binaryflow.block import BiRealNet
from binaryflow.layers.normalization import *
from contextlib import redirect_stdout

In [4]:
  data_format="channels_last"
    
(X_train,y_train),(X_test,y_test)=tf.keras.datasets.mnist.load_data()
X_train=X_train.astype(dtype=np.float32)
y_train=tf.one_hot(y_train,10)

# All quantized layers except the first will use the same options

abc_args = dict(
              kernel_quantizers="ste_sign",
              input_quantizers="ste_sign",
              kernel_constraint="weight_clip",
              #kernel_params={"mu_initializer":tf.keras.initializers.RandomNormal(0,0.05)},
              use_bias=False
              )

bnn_args=dict(kernel_quantizer="ste_sign",
              input_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              use_bias=False)

Args=[bnn_args]*2+[abc_args]

Archs=[BinaryNet.QuantDense,XnorNet.ScaledQuantDense,ABCNet.ABCDense]
Names=["BinaryNet","XnorNet","ABCNet"]



Dense=tf.keras.layers.Dense
Conv2D=ABCNet.ABCConv2D

models=[]

model=tf.keras.models.Sequential([
tf.keras.layers.Flatten(),
#tf.keras.layers.GaussianNoise(stddev=4),
ImageNormalizationLayer(),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(1024,activation="relu"),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(1024,activation="relu"),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(10),
tf.keras.layers.Activation("softmax")
])
models.append(model)
model.compile(
tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
loss="categorical_crossentropy",
metrics=["accuracy"],
)


trained_model = model.fit(
    X_train, 
    y_train,
    batch_size=128, 
    epochs=1,
    validation_data=(X_test, tf.one_hot(y_test, 10)),
    shuffle=True
)

            



/home/ramizouari/anaconda3/envs/tensorflow_2_9/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


469/469 [==============================] - 5s 7ms/step - loss: 0.3110 - accuracy: 0.9230 - val_loss: 3.7355 - val_accuracy: 0.4000


In [21]:
import json
json.dump(model.history.history,open("results/MNIST/ABCNet-Results-MNIST.","w"))

In [5]:
lq.models.summary(model)

+sequential stats--------------------------------------------------------------------+
| Layer                      Input prec.     Outputs  # 32-bit   Memory  32-bit MACs |
|                                  (bit)                   x 1     (kB)              |
+------------------------------------------------------------------------------------+
| flatten                              -   (-1, 784)         0        0            0 |
| image_normalization_layer            -   (-1, 784)         0        0            ? |
| batch_normalization                  -   (-1, 784)      1568     6.12            0 |
| dense                                -  (-1, 1024)    803840  3140.00       802816 |
| batch_normalization_1                -  (-1, 1024)      2048     8.00            0 |
| dense_1                              -  (-1, 1024)   1049600  4100.00      1048576 |
| batch_normalization_2                -  (-1, 1024)      2048     8.00            0 |
| dense_2                              -   

In [10]:

with open("summary.txt","w") as file:
        with redirect_stdout(file):
            for model,name in zip(models,Names):
                print(f"Summary of MNIST results with model {name}")
                lq.models.summary(model)
                print("-------------------------------------------")

In [8]:
print(.op_count("mac"))

None


In [33]:
lq.models.op_count_supported_layer_types=tuple(list(lq.models.op_count_supported_layer_types)+[ABCNet.ABCDense])

In [6]:
isinstance(model.layers[3].estimators[0], lq.models.op_count_supported_layer_types)

True

In [43]:
lq.models.mac_containing_layers

(larq.layers.QuantConv2D,
 larq.layers.QuantSeparableConv2D,
 larq.layers.QuantDepthwiseConv2D,
 larq.layers.QuantDense,
 keras.layers.convolutional.conv2d.Conv2D,
 keras.layers.convolutional.separable_conv2d.SeparableConv2D,
 keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D,
 keras.layers.core.dense.Dense,
 larq.layers.QuantConv1D,
 larq.layers.QuantSeparableConv1D,
 keras.layers.convolutional.conv1d.Conv1D,
 keras.layers.convolutional.separable_conv1d.SeparableConv1D)

In [45]:
lq.models.op_count_supported_layer_types

(keras.layers.reshaping.flatten.Flatten,
 keras.layers.normalization.batch_normalization.BatchNormalization,
 keras.layers.pooling.max_pooling2d.MaxPooling2D,
 keras.layers.pooling.average_pooling2d.AveragePooling2D,
 keras.layers.pooling.max_pooling1d.MaxPooling1D,
 keras.layers.pooling.average_pooling1d.AveragePooling1D,
 larq.layers.QuantConv2D,
 larq.layers.QuantSeparableConv2D,
 larq.layers.QuantDepthwiseConv2D,
 larq.layers.QuantDense,
 keras.layers.convolutional.conv2d.Conv2D,
 keras.layers.convolutional.separable_conv2d.SeparableConv2D,
 keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D,
 keras.layers.core.dense.Dense,
 larq.layers.QuantConv1D,
 larq.layers.QuantSeparableConv1D,
 keras.layers.convolutional.conv1d.Conv1D,
 keras.layers.convolutional.separable_conv1d.SeparableConv1D,
 binaryflow.layers.ABCNet.ABCDense)

In [9]:
profiler=lq.models.LayerProfile(model.layers[3].estimators[0])

In [12]:
profiler.output_shape

In [13]:
model.layers[3].estimators[0].output_shape

AttributeError: The layer "scaled_quant_dense" has never been called and thus has no defined output shape.

In [5]:
isinstance(,tf.keras.Model)

True

In [6]:
s=tf.keras.Sequential([])

In [7]:
s

In [8]:
type(s)

keras.engine.sequential.Sequential